In [1]:
!pip install pulp
!pip install yfinance
!pip install cvxpy
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pulp
import scipy.linalg as la
import yfinance as yf

### Selected stock

In [6]:

# Define the symbols for stocks and bonds
assets = ["EA", "MSFT", "NFLX", "GOOGL", "IGSB", "RYSE", "XBIL"]

# Fetch historical data for each symbol
data = {}
for symbol in assets:
    data[symbol] = yf.download(symbol, period="1y")

# Create a DataFrame to store the close prices
close_prices = pd.DataFrame(columns=assets)

# Populate the DataFrame with close prices
for symbol in assets:
    close_prices[symbol] = data[symbol]['Close']

# Display the DataFrame
print(close_prices)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                    EA        MSFT        NFLX       GOOGL       IGSB  \
Date                                                                    
2023-04-11  125.680000  282.829987  338.209991  105.349998  50.509998   
2023-04-12  126.129997  283.489990  331.029999  104.639999  50.560001   
2023-04-13  128.279999  289.839996  346.190002  107.430000  50.660000   
2023-04-14  127.870003  286.140015  338.630005  108.870003  50.529999   
2023-04-17  128.080002  288.799988  332.720001  105.970001  50.439999   
...                ...         ...         ...         ...        ...   
2024-04-04  129.559998  417.880005  617.140015  150.529999  51.080002   
2024-04-05  130.289993  425.519989  636.179993  152.500000  51.009998   
2024-04-08  130.000000  424.589996  628.409973  154.850006  50.950001   
2024-04-09  130.690002  426.279999  618.200012  156.600006  51.009998   
2024-04-10  129.300003  423.260010  618.580017  156.139999  50.720001   

                 RYSE       XBIL  
Date           

In [16]:
returns = {}
for symbol in symbols:
    close = close_prices[symbol].values
    T = 1  # Number of days in time period
    returns[symbol] = (close[T:] - close[:-T]) / close[:-T]

# Convert returns dictionary to DataFrame
R = pd.DataFrame(returns)

# Display the returns DataFrame
print(R)

           EA      MSFT      NFLX     GOOGL      IGSB      RYSE      XBIL
0    0.003580  0.002334 -0.021229 -0.006739  0.000990 -0.000584  0.000200
1    0.017046  0.022399  0.045796  0.026663  0.001978  0.006625  0.000399
2   -0.003196 -0.012766 -0.021838  0.013404 -0.002566  0.009253 -0.000199
3    0.001642  0.009296 -0.017453 -0.026637 -0.001781  0.011930  0.000000
4    0.007495 -0.001489  0.002945 -0.013872  0.000198 -0.002843  0.000000
..        ...       ...       ...       ...       ...       ...       ...
246 -0.022336 -0.006113 -0.020537 -0.028337  0.000784 -0.005019  0.000801
247  0.005634  0.018283  0.030852  0.013087 -0.001370  0.016652  0.000000
248 -0.002226 -0.002186 -0.012214  0.015410 -0.001176  0.004518  0.000000
249  0.005308  0.003980 -0.016247  0.011301  0.001178 -0.012129  0.000000
250 -0.010636 -0.007085  0.000615 -0.002937 -0.005685  0.035857  0.000100

[251 rows x 7 columns]


In [17]:
r = R.mean()

# Display the mean return for each symbol
print("Mean Return for Each Symbol:")
print(mean_return)

Mean Return for Each Symbol:
EA       0.000176
MSFT     0.001702
NFLX     0.002652
GOOGL    0.001716
IGSB     0.000019
RYSE     0.000094
XBIL    -0.000012
dtype: float64


In [18]:
# Compute covariance between each symbol
S = np.cov(R.T)

# Display the covariance matrix
print("Covariance Matrix:")
print(cov)

Covariance Matrix:
[[ 1.24913550e-04  3.88965376e-05  4.02126633e-05  3.79564600e-05
   2.34632652e-06 -8.88397020e-06 -2.47731611e-07]
 [ 3.88965376e-05  1.90349092e-04  1.22112322e-04  9.89474343e-05
   1.11349563e-06  4.66047021e-06 -6.32001423e-07]
 [ 4.02126633e-05  1.22112322e-04  5.03095045e-04  1.14351464e-04
   8.19450062e-06 -1.20940977e-05 -6.95216801e-07]
 [ 3.79564600e-05  9.89474343e-05  1.14351464e-04  2.93251399e-04
   4.15295237e-06 -5.74052646e-06 -2.25096133e-07]
 [ 2.34632652e-06  1.11349563e-06  8.19450062e-06  4.15295237e-06
   4.08689619e-06 -1.64084785e-05  6.22650425e-07]
 [-8.88397020e-06  4.66047021e-06 -1.20940977e-05 -5.74052646e-06
  -1.64084785e-05  2.32413297e-04 -7.24634137e-07]
 [-2.47731611e-07 -6.32001423e-07 -6.95216801e-07 -2.25096133e-07
   6.22650425e-07 -7.24634137e-07  8.99136718e-07]]


## Model 1A

In [20]:
from pulp import *
import numpy as np

# Define the data
n = len(assets)  # Number of assets
T = len(R)  # Number of time periods
r0 = 0.0008

# Define returns and covariance
R = R.values
S = np.cov(R.T)
r = mean_return.values

# Create a LP problem
prob = LpProblem("Financial_Portfolio_Optimization", LpMaximize)

# Define decision variables
x = [LpVariable(f"x{j}", lowBound=0) for j in range(n)]
y = [LpVariable(f"y{t}", lowBound=0) for t in range(T)]

# Define objective function
prob += -(1 / T) * lpSum(y[t] for t in range(T)), "Objective"

# Define constraints
for t in range(T):
    prob += (
        lpSum(x[j] * (R[t][j] - r[j]) for j in range(n)) <= y[t],
        f"Upper_constraint_{t}",
    )
    prob += (
        lpSum(x[j] * (R[t][j] - r[j]) for j in range(n)) >= -y[t],
        f"Lower_constraint_{t}",
    )

prob += lpSum(r[j] * x[j] for j in range(n)) == r0

prob += lpSum(x) == 1, "Sum_of_weights"

# Solve the problem
prob.solve()

# Print the results
print("Status:", LpStatus[prob.status])
print("Optimal Portfolio Weights:")
for j in range(n):
    print(f"x{j}: {value(x[j])}")
print("Expected Return (MAD):", value(lpSum(r[j] * x[j] for j in range(n))))
print("Mean Absolute Deviation (MAD):", value((1 / T) * lpSum(y[t] for t in range(T))))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1bcc839fb0de41db81029ce709134221-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/1bcc839fb0de41db81029ce709134221-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 509 COLUMNS
At line 4791 RHS
At line 5296 BOUNDS
At line 5297 ENDATA
Problem MODEL has 504 rows, 258 columns and 4030 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 497 (-7) rows, 258 (0) columns and 3974 (-56) elements
Perturbing problem by 0.001% of 0.0053852038 - largest nonzero change 4.9871207e-05 ( 17.070369%) - largest zero change 4.0815471e-05
0  Obj -0 Primal inf 0.96866049 (2)
39  Obj -0.00090853638 Primal inf 205.04533 (228)
85  Obj -0.0017096251 Primal inf 250.57879 (231)
136  Obj -0.0028571806 Primal inf 134.45391 (177)
187  Obj -0.00363653

## Model 1B

In [21]:
from pulp import *

# Define the data
n = len(assets)  # Number of assets
T = len(R[:, 0])  # Number of time periods
m0 = 0.007

# Create a LP problem
prob = LpProblem("Financial_Portfolio_Optimization", LpMaximize)

# Define decision variables
x = [LpVariable(f"x{j}", lowBound=0) for j in range(n)]
y = [LpVariable(f"y{t}", lowBound=0) for t in range(T)]

# Define objective function
prob += lpSum(r[j] * x[j] for j in range(n)), "Objective"

# Define constraints
for t in range(T):
    prob += (
        lpSum(x[j] * (R[t][j] - r[j]) for j in range(n)) <= y[t],
        f"Upper_constraint_{t}",
    )
    prob += (
        lpSum(x[j] * (R[t][j] - r[j]) for j in range(n)) >= -y[t],
        f"Lower_constraint_{t}",
    )

prob += (1 / T) * lpSum(y[t] for t in range(T)) <= m0, "Risk"

prob += lpSum(x) == 1, "Sum_of_weights"

# Solve the problem
prob.solve()

# Print the results
print("Status:", LpStatus[prob.status])
print("Optimal Portfolio Weights:")
for j in range(n):
    print(f"x{j}: {value(x[j])}")
print("Expected Return (MAD):", value(lpSum(r[j] * x[j] for j in range(n))))
print("Mean Absolute Deviation (MAD):", value((1 / T) * lpSum(y[t] for t in range(T))))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/afb19688794141559cf2f73145705f61-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/afb19688794141559cf2f73145705f61-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 509 COLUMNS
At line 4791 RHS
At line 5296 BOUNDS
At line 5297 ENDATA
Problem MODEL has 504 rows, 258 columns and 4274 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 497 (-7) rows, 258 (0) columns and 4218 (-56) elements
Perturbing problem by 0.001% of 0.0052032367 - largest nonzero change 0.00011908808 ( 34.093615%) - largest zero change 9.9715536e-05
0  Obj -0 Primal inf 14.559002 (2) Dual inf 0.015417065 (6)
64  Obj 0.004252521 Primal inf 1991.1969 (231)
95  Obj 0.002139026 Primal inf 364.64994 (203)
137  Obj 0.0013704446 Primal inf 63.343826 (134)
18

### Model 1C

In [22]:
from pulp import *

# Define the data
n = len(assets)  # Number of assets
T = len(R[:, 0])  # Number of time periods
mu = 1

# Create a LP problem
prob = LpProblem("Financial_Portfolio_Optimization", LpMaximize)

# Define decision variables
x = [LpVariable(f"x{j}", lowBound=0) for j in range(n)]
y = [LpVariable(f"y{t}", lowBound=0) for t in range(T)]

# Define objective function
prob += lpSum(r[j] * x[j] for j in range(n)) * mu - (1 / T) * lpSum(y[t] for t in range(T)), "Objective"

# Define constraints
for t in range(T):
    prob += lpSum(x[j] * (R[t][j] - r[j]) for j in range(n)) <= y[t], f"Upper_constraint_{t}"
    prob += lpSum(x[j] * (R[t][j] - r[j]) for j in range(n)) >= -y[t], f"Lower_constraint_{t}"

prob += lpSum(x) == 1, "Sum_of_weights"

# Solve the problem
prob.solve()

# Print the results
print("Status:", LpStatus[prob.status])
print("Optimal Portfolio Weights:")
for j in range(n):
    print(f"x{j}: {value(x[j])}")
print("Expected Return (MAD):", value(lpSum(r[j] * x[j] for j in range(n))))
print("Mean Absolute Deviation (MAD):", value((1 / T) * lpSum(y[t] for t in range(T))))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/22d338083ef94873914b8ef16af6a7c1-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/22d338083ef94873914b8ef16af6a7c1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 508 COLUMNS
At line 4790 RHS
At line 5294 BOUNDS
At line 5295 ENDATA
Problem MODEL has 503 rows, 258 columns and 4023 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 496 (-7) rows, 258 (0) columns and 3967 (-56) elements
Perturbing problem by 0.001% of 0.0053852038 - largest nonzero change 8.167337e-05 ( 23.50228%) - largest zero change 0
0  Obj -0 Primal inf 0.18010583 (1) Dual inf 0.015603863 (6)
58  Obj -0.00019713755 Primal inf 84.242691 (215)
114  Obj -0.00030226203 Primal inf 13.135137 (198)
168  Obj -0.00036722229 Primal inf 6.3355458 (143)
218  O

### Model 2A

In [23]:
import cvxpy as cp
import numpy as np

# Define the data
R_c = R - r.reshape(1, -1)
sig = np.dot(R_c.T, R_c) / T
r0 = 0.0005
n = len(R[0, :])
e = np.ones((n, 1))
rq = r.reshape(n, 1)

# Define the optimization variables
X = cp.Variable((n, 1))

# Define the objective function
Objective = cp.Minimize(cp.quad_form(X, sig))

# Define the constraints
constraints = [e.T @ X == 1, X >= 0, rq.T @ X >= r0]

# Formulate the optimization problem
problem = cp.Problem(Objective, constraints)

# Solve the problem
result = problem.solve()

# Print the results
print("Optimal Portfolio Weights:")
print(X.value.T)
print("Expected Return:", (rq.T @ X.value)[0])
print("Mean Variance:", (X.value.T @ sig @ X.value)[0])

Optimal Portfolio Weights:
[[6.25649663e-15 1.20880508e-01 7.42359910e-02 6.08233316e-02
  9.75002844e-04 1.73353617e-02 7.25749805e-01]]
Expected Return: [0.0005]
Mean Variance: [1.15698837e-05]


### Model 2B

In [24]:
import cvxpy as cp
import numpy as np

# Define the data
n = len(assets)
q = 0.1
e = np.ones(n)

# Define the optimization variables
X = cp.Variable(n)

# Define the objective function
obj = cp.Minimize(cp.quad_form(X, S) - q * r @ X)

# Define the constraints
constraints = [e @ X == 1, X >= 0]

# Formulate the optimization problem
problem = cp.Problem(obj, constraints)

# Solve the problem
result = problem.solve()

# Print the optimal portfolio weights
print("Optimal Portfolio Weights:")
print(X.value)

Optimal Portfolio Weights:
[-1.97546506e-23  2.70351476e-01  1.68992346e-01  1.38446682e-01
  1.11124008e-23  3.08501058e-02  3.91359391e-01]


### Model 2C

In [25]:
import numpy as np
import numpy.linalg as la

# Define the data
n = len(assets)
e = np.ones((n, 1))
r = r.reshape(n, 1)
mu = 0.001

# Construct the matrix M and vector b
M = np.block([[2 * S, -r, -e], [r.T, 0, 0], [e.T, 0, 0]])
b = np.block([[np.zeros((n, 1))], [mu], [1]])

# Solve for optimal portfolio weights
x = la.solve(M, b)
print(x)


[[-9.19562364e-02]
 [ 2.45234692e-01]
 [ 1.47700303e-01]
 [ 1.24903886e-01]
 [-8.08036706e-02]
 [ 1.90495016e-02]
 [ 6.35871524e-01]
 [ 8.50239333e-02]
 [ 1.50045804e-06]]
